In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

from jpasLAEs.load_jpas_catalogs import load_minijpas_jnep
from load_paus_cat import load_paus_cat
from paus_utils import w_central
from jpasLAEs.utils import flux_to_mag

from astropy.coordinates import SkyCoord
import astropy.units as u

import numpy as np

In [ ]:
# Load miniJPAS
mj_cat_dir = '/home/alberto/cosmos/LAEs/csv'
mj_cat = load_minijpas_jnep(mj_cat_dir)

# Load PAUS
field_name = 'W3'
path_to_cat = [f'/home/alberto/almacen/PAUS_data/catalogs/PAUS_3arcsec_{field_name}_extinction_corrected.pq']
paus_cat = load_paus_cat(path_to_cat)

mask_NB_number = (paus_cat['NB_number'] > 39)
paus_cat['flx'] = paus_cat['flx'][:, mask_NB_number]
paus_cat['err'] = paus_cat['err'][:, mask_NB_number]
paus_cat['NB_mask'] = paus_cat['NB_mask'][:, mask_NB_number]
for key in paus_cat.keys():
    if key in ['flx', 'err', 'NB_mask', 'area']:
        continue
    paus_cat[key] = paus_cat[key][mask_NB_number]

stack_nb_ids = np.arange(12, 26 + 1)
synth_BB_flx = np.average(paus_cat['flx'][stack_nb_ids],
                          weights=paus_cat['err'][stack_nb_ids] ** -2,
                          axis=0)

paus_cat['synth_r_mag'] = flux_to_mag(synth_BB_flx, w_central[-4])
paus_cat['r_mag'] = flux_to_mag(paus_cat['flx'][-4], w_central[-4])

In [ ]:
# Xmatch
coords_paus = SkyCoord(ra=paus_cat['RA'] * u.deg,
                       dec=paus_cat['DEC'] * u.deg)
coords_mj = SkyCoord(ra=mj_cat['RA'] * u.deg,
                       dec=mj_cat['DEC'] * u.deg)
                    
xm_id, ang_dist, _= coords_paus.match_to_catalog_sky(coords_mj)

# Objects with 1 arcsec of separation
mask_dist = (ang_dist <= 1 * u.arcsec)

print(f'Matched: {sum(mask_dist)}')

In [ ]:
# Select LAEs
from LAE_selection_method import select_LAEs

nb_min, nb_max = 0, 20
r_min, r_max = 17, 24

paus_cat = select_LAEs(paus_cat, nb_min, nb_max, r_min, r_max,
                       ew0min_lya=20, ewmin_other=100)

In [ ]:
import pandas as pd

mj_sel_dir = '/home/alberto/cosmos/LAEs/csv/selection.csv'
mj_selection = pd.read_csv(mj_sel_dir)
mj_selection.head()

In [ ]:
from jpasLAEs.utils import z_NB as jpas_z_NB

# Add stuff to mj_cat
mj_cat['L_lya'] = np.zeros_like(mj_cat['number']).astype(float)
mj_cat['z_NB'] = np.zeros_like(mj_cat['number']).astype(float)
mj_cat['nice_lya'] = np.zeros_like(mj_cat['number']).astype(bool)

for i, src in enumerate(mj_selection['src']):
    mj_cat['L_lya'][src] = mj_selection['L_lya'][i]
    mj_cat['z_NB'][src] = jpas_z_NB(mj_selection['nb_sel'][i])
    mj_cat['nice_lya'][src] = True

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

L_bins = np.linspace(42, 46, 20)

ax.hist(paus_cat['L_lya'][paus_cat['nice_lya']],
        histtype='step', label='PAUS W3',
        density=True)
ax.hist(mj_cat['L_lya'][mj_cat['nice_lya']],
        histtype='step', label='miniJPAS',
        density=True)

ax.legend(fontsize=14)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

ax.scatter(paus_cat['z_NB'], mj_cat['z_NB'][xm_id])

xx = [-100, 100]
ax.plot(xx, xx, ls='--', c='r')

ax.set_xlabel('Redshift (PAUS)')
ax.set_ylabel('Redshift (miniJPAS)')
ax.set_xlim(2, 5)
ax.set_ylim(2, 5)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

mask = np.abs(paus_cat['z_NB'] - mj_cat['z_NB'][xm_id]) < 0.2
ax.scatter(paus_cat['L_lya'][mask], mj_cat['L_lya'][xm_id][mask])

xx = [-100, 100]
ax.plot(xx, xx, ls='--', c='r')

ax.set_xlabel('L_lya (PAUS)')
ax.set_ylabel('L_lya (miniJPAS)')
ax.set_xlim(43, 45.5)
ax.set_ylim(43, 45.5)

plt.show()